### **Phase 1: Initial Data Assessment & Cleaning**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Setting display options to see more data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

#### **STEP 1.1: Load and Inspect Dataset**

In [2]:
df = pd.read_csv("Airbnb_Open_Data.csv")

In [3]:
print("No of Rows:", len(df))


No of Rows: 102599


In [4]:
print("No of Columns:", len(df.columns))


No of Columns: 26


In [5]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,country code,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,US,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,US,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  str    
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  str    
 4   host name                       102193 non-null  str    
 5   neighbourhood group             102570 non-null  str    
 6   neighbourhood                   102583 non-null  str    
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  str    
 10  country code                    102468 non-null  str    
 11  instant_bookable                102494 non-null  object 
 12  cancellation_policy        

#### **STEP 1.2: Check Percentage of Null Values in Each Column**

In [7]:
null_percentage = (df.isnull().sum() / len(df) * 100).round(2)
print(null_percentage)


id                                  0.00
NAME                                0.24
host id                             0.00
host_identity_verified              0.28
host name                           0.40
neighbourhood group                 0.03
neighbourhood                       0.02
lat                                 0.01
long                                0.01
country                             0.52
country code                        0.13
instant_bookable                    0.10
cancellation_policy                 0.07
room type                           0.00
Construction year                   0.21
price                               0.24
service fee                         0.27
minimum nights                      0.40
number of reviews                   0.18
last review                        15.49
reviews per month                  15.48
review rate number                  0.32
calculated host listings count      0.31
availability 365                    0.44
house_rules     

#### **STEP 1.2: Check & Remove Duplicates**

In [8]:
complete_duplicates = df.duplicated().sum()
print(f"Complete duplicate rows: {complete_duplicates}")
print(f"Total rows: {len(df)}")

Complete duplicate rows: 541
Total rows: 102599


In [9]:
df = df.drop_duplicates()
print(f"Rows after removing duplicates: {len(df)}")

Rows after removing duplicates: 102058


#### **STEP 1.4: Dropping the columns with more then 50% null values & non useful columns**

In [10]:
df = df.drop(columns=['license', 'house_rules', 'instant_bookable', 'reviews per month', 'country', 'country code'])
print(f"Remaining columns: {len(df.columns)}")
print(df.columns.tolist())

Remaining columns: 20
['id', 'NAME', 'host id', 'host_identity_verified', 'host name', 'neighbourhood group', 'neighbourhood', 'lat', 'long', 'cancellation_policy', 'room type', 'Construction year', 'price', 'service fee', 'minimum nights', 'number of reviews', 'last review', 'review rate number', 'calculated host listings count', 'availability 365']


#### **STEP 1.5: Fixing Data types**

**STEP 1.5.1: Fixing the Price Column**

In [11]:
print(df['price'].dtype)
print(df['price'].head())

str
0    $966 
1    $142 
2    $620 
3    $368 
4    $204 
Name: price, dtype: str


In [12]:
df['price'] = df['price'].str.replace('$', '').str.replace(',', '')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

#finding if any null values are there or not after the operation
print(f"Any nulls created: {df['price'].isnull().sum()}")

Any nulls created: 247


In [13]:
df = df[df['price'].notna()]
df = df[df['price'] > 0]

In [14]:
# Check extreme prices
print(f"Zero prices: {(df['price'] == 0).sum()}")
print(f"Prices > $1,200: {(df['price'] > 1200).sum()}")

Zero prices: 0
Prices > $1,200: 0


**STEP 1.5.2: Fixing the Service Fee Column**

In [15]:

df['service fee'] = df['service fee'].str.replace('$', '').str.replace(',', '')
df['service fee'] = pd.to_numeric(df['service fee'], errors='coerce')
df['service fee'] = df['service fee'].fillna(0)

In [16]:
# Check extreme prices
print(f"Zero service fee: {(df['service fee'] == 0).sum()}")
print(f"Service Fee > $400: {(df['service fee'] > 400).sum()}")

Zero service fee: 239
Service Fee > $400: 0


**STEP 1.5.3: Fixing the Last review Column**

In [17]:

df['last review'] = pd.to_datetime(df['last review'], errors='coerce')
print(f"Null Values: {df['last review'].isnull().sum()} (listings with no reviews)")


Null Values: 15806 (listings with no reviews)


In [18]:
# dropping last review column
df = df.drop(columns=['last review']) 

**STEP 1.5.4: Fixing the Construction_year Column**

In [19]:
# Convert Construction year to integer
df['Construction year'] = pd.to_numeric(df['Construction year'], errors='coerce')
df.loc[(df['Construction year'] < 1800) | (df['Construction year'] > 2025), 'Construction year'] = np.nan
print(f"Null Values: {df['Construction year'].isnull().sum()}")

Null Values: 210


**STEP 1.5.5: Checking the Neighborhood & Neighborhood group Columns**
This will help us in checking if any listings are missing their location (neighbourhood or borough).
Location is critical for visualization, we can't map or analyze listings without knowing where they are.


In [20]:
print(f"Neighbourhood nulls: {df['neighbourhood'].isnull().sum()}")
print(f"Neighbourhood group nulls: {df['neighbourhood group'].isnull().sum()}")

Neighbourhood nulls: 15
Neighbourhood group nulls: 28


In [21]:
df = df[df['neighbourhood'].notna()]
df = df[df['neighbourhood group'].notna()]

#### **STEP 1.6: Standardising the Text Columns**

**STEP 1.6.1: Standardizing the Room type, neighborhood & neighborhood group**

In [22]:
# Standardize text - remove extra spaces and fix capitalization
df['room type'] = df['room type'].str.strip().str.title()
df['neighbourhood group'] = df['neighbourhood group'].str.strip().str.title()
df['neighbourhood'] = df['neighbourhood'].str.strip()


In [23]:
print(f"\nRoom types: {df['room type'].unique()}")
print(f"Boroughs: {df['neighbourhood group'].unique()}")


Room types: <ArrowStringArray>
['Private Room', 'Entire Home/Apt', 'Shared Room', 'Hotel Room']
Length: 4, dtype: str
Boroughs: <ArrowStringArray>
['Brooklyn', 'Manhattan', 'Brookln', 'Manhatan', 'Queens', 'Staten Island', 'Bronx']
Length: 7, dtype: str


In [24]:
#fixing the spelling mistakes in the borough
df['neighbourhood group'] = df['neighbourhood group'].replace({
    'Brookln': 'Brooklyn',
    'Manhatan': 'Manhattan'
})

print(f"Boroughs after fix: {df['neighbourhood group'].unique()}")

Boroughs after fix: <ArrowStringArray>
['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx']
Length: 5, dtype: str


**STEP 1.6.2: Validating the Geographical Data**
Check if all listings are within NYC boundaries:

In [25]:
# NYC boundaries (approximate)
NYC_LAT_MIN, NYC_LAT_MAX = 40.5, 40.95
NYC_LONG_MIN, NYC_LONG_MAX = -74.3, -73.7

# Find listings outside NYC
out_of_bounds = ((df['lat'] < NYC_LAT_MIN) | (df['lat'] > NYC_LAT_MAX) |
                 (df['long'] < NYC_LONG_MIN) | (df['long'] > NYC_LONG_MAX)).sum()

print(f"Listings outside NYC boundaries: {out_of_bounds}")

if out_of_bounds > 0:
    # Remove them
    df = df[(df['lat'] >= NYC_LAT_MIN) & (df['lat'] <= NYC_LAT_MAX) &
            (df['long'] >= NYC_LONG_MIN) & (df['long'] <= NYC_LONG_MAX)]
    print(f"Removed {out_of_bounds} listings")
    print(f"Rows remaining: {len(df)}")
else:
    print("All listings within NYC boundaries")

Listings outside NYC boundaries: 1
Removed 1 listings
Rows remaining: 101759


**STEP 1.6.3: Check review rating (should be 0-5)**


In [26]:
invalid_rating = ((df['review rate number'] < 0) | (df['review rate number'] > 5)).sum()
print(f"Invalid review ratings: {invalid_rating}")


Invalid review ratings: 0


**STEP 1.6.4: Check minimum nights**


In [27]:
invalid_nights = ((df['minimum nights'] < 1)).sum()
print(f"Invalid minimum nights: {invalid_nights}")

Invalid minimum nights: 13


**STEP 1.6.5:  Check availability (should be 0-365)**


In [28]:
invalid_avail = ((df['availability 365'] < 0) | (df['availability 365'] > 365)).sum()
print(f"Invalid availability values: {invalid_avail}")

Invalid availability values: 3170


**STEP 1.6.6: FIXING THE INVALID DATA FROM STEP 1.6.4 & 1.6.5**

In [29]:
df = df[df['minimum nights'] >= 1]
# Fix availability (cap at 365, set negatives to 0)
df.loc[df['availability 365'] < 0, 'availability 365'] = 0
df.loc[df['availability 365'] > 365, 'availability 365'] = 365
df['availability 365'] = df['availability 365'].fillna(0)


**STEP 1.6.6: Checking the nulls and imputing the host_identity_verified**

In [30]:
df['host_identity_verified'] = df['host_identity_verified'].fillna('Unknown')
df['host_identity_verified'] = df['host_identity_verified'].replace({
    'unconfirmed': 'Not Verified',
    'Unknown': 'Not Verified',
    'verified': 'Verified'
})

print(df['host_identity_verified'].value_counts())

host_identity_verified
Not Verified    50880
Verified        50468
Name: count, dtype: int64


**STEP 1.6.6: Checking the nulls and imputing the host_name**

In [31]:
df['host name'] = df['host name'].fillna('Unknown')

**STEP 1.6.7: Imputing the remaining nulls**

In [32]:
df['NAME'] = df['NAME'].fillna('Unnamed Listing')
df['cancellation_policy'] = df['cancellation_policy'].fillna('unknown')
df['number of reviews'] = df['number of reviews'].fillna(0)
df['review rate number'] = df['review rate number'].fillna(0)
df['calculated host listings count'] = df['calculated host listings count'].fillna(1)

print(f"Total nulls remaining: {df.isnull().sum().sum()}")

Total nulls remaining: 193


#### **STEP 2: Checking the final nulls**

In [33]:
for col in df.columns:
    nulls = df[col].isnull().sum()
    print(f"{col}: {nulls} nulls")

id: 0 nulls
NAME: 0 nulls
host id: 0 nulls
host_identity_verified: 0 nulls
host name: 0 nulls
neighbourhood group: 0 nulls
neighbourhood: 0 nulls
lat: 0 nulls
long: 0 nulls
cancellation_policy: 0 nulls
room type: 0 nulls
Construction year: 193 nulls
price: 0 nulls
service fee: 0 nulls
minimum nights: 0 nulls
number of reviews: 0 nulls
review rate number: 0 nulls
calculated host listings count: 0 nulls
availability 365: 0 nulls


### **PHASE 2: Create Calculated Columns**

**STEP 1: Create total_price**

In [34]:
df['total_price'] = df['price'] + df['service fee']
print(f"Range: ${df['total_price'].min()} - ${df['total_price'].max()}")

Range: $50.0 - $1440.0


**STEP 2:  Create price_category**

In [35]:
df['price_category'] = pd.cut(df['price'], 
                               bins=[0, 100, 250, 500, float('inf')],
                               labels=['Budget', 'Mid-Range', 'Premium', 'Luxury'])

print(df['price_category'].value_counts())

price_category
Luxury       61734
Premium      21983
Mid-Range    13191
Budget        4440
Name: count, dtype: int64


In [36]:
print(df.columns.tolist())


['id', 'NAME', 'host id', 'host_identity_verified', 'host name', 'neighbourhood group', 'neighbourhood', 'lat', 'long', 'cancellation_policy', 'room type', 'Construction year', 'price', 'service fee', 'minimum nights', 'number of reviews', 'review rate number', 'calculated host listings count', 'availability 365', 'total_price', 'price_category']


**STEP 3: Creating host_type from calculated host listings count**
Creating host_type based on listing count

In [37]:
df['host_type'] = pd.cut(df['calculated host listings count'],
                          bins=[0, 1, 5, 20, float('inf')],
                          labels=['Single', 'Small (2-5)', 'Medium (6-20)', 'Large (20+)'])

print(df['host_type'].value_counts())

host_type
Single           63022
Small (2-5)      26237
Medium (6-20)     6403
Large (20+)       5686
Name: count, dtype: int64


**STEP 4: Creating Availability category from availability 365**

In [38]:
df['availability_category'] = pd.cut(df['availability 365'],
                                     bins=[-1, 89, 179, 364, 365],
                                     labels=['Low (<90)', 'Medium (90-179)', 
                                            'High (180-364)', 'Always (365)'])
print(df['availability_category'].value_counts())

availability_category
Low (<90)          49351
High (180-364)     31912
Medium (90-179)    14892
Always (365)        5193
Name: count, dtype: int64


**STEP 5: Creating listing quality from number of reviews & ratings**


In [39]:
# Create simple listing_quality column
df['listing_quality'] = 'Unrated'

# Excellent (4.5+ rating with reviews)
df.loc[(df['review rate number'] >= 4.5) & (df['number of reviews'] >= 5), 'listing_quality'] = 'Excellent'

# Good (4.0-4.5 rating with reviews)
df.loc[(df['review rate number'] >= 4.0) & (df['review rate number'] < 4.5) & (df['number of reviews'] >= 5), 'listing_quality'] = 'Good'

# Poor (<4.0 rating with reviews)
df.loc[(df['review rate number'] < 4.0) & (df['number of reviews'] >= 5), 'listing_quality'] = 'Poor'

print(df['listing_quality'].value_counts())

listing_quality
Unrated      42525
Poor         32197
Excellent    13426
Good         13200
Name: count, dtype: int64


In [40]:
df.head(20)

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,review rate number,calculated host listings count,availability 365,total_price,price_category,host_type,availability_category,listing_quality
0,1001254,Clean & quiet apt home by the park,80014485718,Not Verified,Madaline,Brooklyn,Kensington,40.64749,-73.97237,strict,Private Room,2020.0,966.0,193.0,10.0,9.0,4.0,6.0,286.0,1159.0,Luxury,Medium (6-20),High (180-364),Good
1,1002102,Skylit Midtown Castle,52335172823,Verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,moderate,Entire Home/Apt,2007.0,142.0,28.0,30.0,45.0,4.0,2.0,228.0,170.0,Mid-Range,Small (2-5),High (180-364),Good
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,Not Verified,Elise,Manhattan,Harlem,40.80902,-73.94190,flexible,Private Room,2005.0,620.0,124.0,3.0,0.0,5.0,1.0,352.0,744.0,Luxury,Single,High (180-364),Unrated
3,1002755,Unnamed Listing,85098326012,Not Verified,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,moderate,Entire Home/Apt,2005.0,368.0,74.0,30.0,270.0,4.0,1.0,322.0,442.0,Premium,Single,High (180-364),Good
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,Verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,moderate,Entire Home/Apt,2009.0,204.0,41.0,10.0,9.0,3.0,1.0,289.0,245.0,Mid-Range,Single,High (180-364),Poor
5,1004098,Large Cozy 1 BR Apartment In Midtown East,45498551794,Verified,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,flexible,Entire Home/Apt,2013.0,577.0,115.0,3.0,74.0,3.0,1.0,365.0,692.0,Luxury,Single,Always (365),Poor
6,1004650,BlissArtsSpace!,61300605564,Not Verified,Alberta,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,moderate,Private Room,2015.0,71.0,14.0,45.0,49.0,5.0,1.0,224.0,85.0,Budget,Single,High (180-364),Excellent
7,1005202,BlissArtsSpace!,90821839709,Not Verified,Emma,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,moderate,Private Room,2009.0,1060.0,212.0,45.0,49.0,5.0,1.0,219.0,1272.0,Luxury,Single,High (180-364),Excellent
8,1005754,Large Furnished Room Near B'way,79384379533,Verified,Evelyn,Manhattan,Hell's Kitchen,40.76489,-73.98493,strict,Private Room,2005.0,1018.0,204.0,2.0,430.0,3.0,1.0,180.0,1222.0,Luxury,Single,High (180-364),Poor
9,1006307,Cozy Clean Guest Room - Family Apt,75527839483,Not Verified,Carl,Manhattan,Upper West Side,40.80178,-73.96723,strict,Private Room,2015.0,291.0,58.0,2.0,118.0,5.0,1.0,365.0,349.0,Premium,Single,Always (365),Excellent


In [41]:
df.to_csv('NYC_Airbnb_Cleaned_Final.csv', index=False)
